In [1]:
import os
import pandas as pd
import csv
import math
import numpy as np
from ipywidgets import Dropdown

In [7]:
folder_path = '..\eyetracking\Assets\Scripts\Data'

def load_data(file):
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path, delimiter=';', usecols=['GazeTime', 'Region', 'Target', 'PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ'])
    data = data[~((data['PosX'] == 0) & (data['PosY'] == 0) & (data['PosZ'] == 0))]

    # Get the unique regions
    regions = data['Region'].unique()

    # Skip the first region
    for region in regions[1:]:
        # Get the index of the first row of the current region
        first_row_index = data[data['Region'] == region].index[0]
        # Drop the first row of the current region
        data = data.drop(first_row_index)

    # Normalize position data using Min-Max normalization
    pos_columns = ['PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ']
    for column in pos_columns:
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

    return data

# Create a list to store the dataframes
dataframes = [load_data(file) for file in os.listdir(folder_path) if file.endswith('.csv')]
dataframes[1]

,GazeTime,Region,Target,PosX,PosY,PosZ,PlayerPosX,PlayerPosY,PlayerPosZ
0,1.626202,1,None,0.093465,0.808150,0.785294,0.000000,0.499486,0.042819
1,0.031551,1,None,0.284908,0.811780,0.602525,0.000000,0.489775,0.058659
2,0.023037,1,None,0.153686,0.784836,0.799078,0.000000,0.482685,0.070225
3,0.066861,1,None,0.285834,0.824554,0.654939,0.000000,0.462106,0.103793
4,0.055239,1,Size(Max),0.292380,0.834804,0.680269,0.000000,0.445104,0.131526
5,0.076259,1,Size(Max),0.267574,0.836015,0.812369,0.000000,0.421632,0.169813
6,0.068541,1,Size(Max),0.340478,0.823588,0.721481,0.000000,0.400537,0.204224
7,0.231608,1,None,0.342710,0.824161,0.815169,0.000000,0.329254,0.320505
8,0.253564,1,None,0.409023,0.804518,0.679536,0.000000,0.251211,0.447809
9,0.085727,1,None,0.445496,0.792184,0.671043,0.000000,0.224826,0.490848


In [3]:
def sum(arr):
    ret = 0
    for i in arr:
        ret += i
    return ret

def PercFixInside(dataframe):
    def Compute(region, interest):
        ret = pd.DataFrame()
        target_names = dataframe[dataframe['Region'] == region]['Target'].unique()
        grouped_data = dataframe[dataframe['Region'] == region].groupby('Target')
        gaze = [0] * len(target_names)  # Initialize list with zeros
        perc = [0] * len(target_names)
        ret['Target'] = target_names
        
        for j, target in enumerate(target_names):
            gaze[j] = grouped_data.get_group(target)[interest].sum()
        tot = sum(gaze)

        for i, g in enumerate(gaze):
            perc[i] = (g / tot) * 100 if tot > 0 else 0

        ret['GazeTime'] = gaze
        ret['Percentage'] = perc
        return ret

    regions = dataframe['Region'].unique()

    # Initialize a list to store the DataFrames for each region
    results = []

    for region in regions:
        newdata = Compute(region, 'GazeTime')
        newdata['Region'] = region  # Add a column for the region
        results.append(newdata)

    # Concatenate all the result DataFrames into one
    final_df = pd.concat(results, ignore_index=True)

    return final_df



In [4]:
def NFix(dataframe, threshold, ShowPercentage = False):
    if(threshold < 0):
        threshold = abs(threshold)
    nfix = {}  # Initialize the NFix counter as a dictionary
    counter = {}  # Initialize the counter as a dictionary
    pos = {}  # Variables to track previous position for each target

    for _, row in dataframe.iterrows():
        region = row['Region']
        target = row['Target']
        if region not in counter:
            counter[region] = {}
            nfix[region] = {}
        if target not in counter[region]:
            counter[region][target] = 0
            nfix[region][target] = 0
            pos[region, target] = (None, None)

        counter[region][target] += 1
        pos_x_new, pos_y_new = float(row['PosX']), float(row['PosY'])  # Get the X and Y positions

        # Check if a fixation occurred by comparing the current position with the previous position
        if pos[region, target][0] is not None and pos[region, target][1] is not None:
            pos_diff = abs(pos[region, target][0] - pos_x_new) + abs(pos[region, target][1] - pos_y_new)
            if pos_diff > threshold:
                nfix[region][target] += 1  # Increment the NFix counter for the target

        pos[region, target] = (pos_x_new, pos_y_new)  # Update the previous position for the target

    if ShowPercentage:
        for region in nfix:
            for target in nfix[region]:
                nfix[region][target] = (nfix[region][target] / counter[region][target]) * 100 if counter[region][target] > 0 else 0

    # Convert nested dictionary to DataFrame
    nfix_df = pd.DataFrame([(reg, tar, val) for reg, tar_val in nfix.items() for tar, val in tar_val.items()], 
                           columns=['Region', 'Target', 'NFix'])

    return nfix_df

In [5]:
def ConvergTime(dataframe):
    # Group the data by region and then by target
    dataframe = dataframe[dataframe['Target'] != 'None']
    grouped_data = dataframe.groupby(['Region', 'Target'])

    # Initialize an empty DataFrame to store the results
    results = pd.DataFrame(columns=['Region', 'Target', 'ConvergenceTime'])

    # Iterate over each group (region and target)
    for (region, target), group in grouped_data:
        # Calculate the average gaze time for the current group (target within the region)
        avg_gaze_time = group['GazeTime'].mean()
        
        # Append the result to the results DataFrame
        results = pd.concat([results, pd.DataFrame({'Region': [region], 'Target': [target], 'ConvergenceTime': [avg_gaze_time]})], ignore_index=True)

    return results

In [6]:
def calculate_distance(df):
    # Calculate the Euclidean distance
    df['Distance'] = np.sqrt((df['PosX'] - df['PlayerPosX'])**2 + (df['PosY'] - df['PlayerPosY'])**2 + (df['PosZ'] - df['PlayerPosZ'])**2)
    # Remove rows with 'None' Target
    df = df[df['Target'] != 'None']
    # Group by 'Target' and calculate mean distance
    df = df.groupby('Target')['Distance'].mean().reset_index()
    return df



In [ ]:
### gaze_data = dataframes[2]
#PercFixInside(gaze_data)
#NFix(gaze_data, 0.05, False)
#ConvergTime(gaze_data)

In [19]:
# Step 1: Apply PercFixInside to each DataFrame in dataframes and accumulate the results
perc_results = [PercFixInside(df) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_perc = pd.concat(perc_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_perc = all_perc.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'GazeTime' and 'Percentage' for each 'Target' in each 'Region'
avg_perc

,Region,Target,GazeTime,Percentage
0,1,None,1.409310,48.576496
1,1,Size(Low),0.435553,22.875180
2,1,Size(Max),0.417627,18.821719
3,1,Size(Mid),0.603867,26.882990
4,2,None,0.869097,52.922662
5,2,Transparency(Low),0.425224,24.011047
6,2,Transparency(Max),0.377899,24.752199
7,2,Transparency(Mid),0.972869,56.295554
8,2,Transparency(Min),0.536326,33.746155
9,3,Black,0.268902,15.608532


In [20]:
# Step 1: Apply NFix to each DataFrame in dataframes and accumulate the results
nfix_results = [NFix(df, 0.05, False) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_nfix = pd.concat(nfix_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_nfix = all_nfix.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'Count' and 'Percentage' for each 'Target' in each 'Region'
avg_nfix

,Region,Target,NFix
0,1,None,5.000000
1,1,Size(Low),2.000000
2,1,Size(Max),1.500000
3,1,Size(Mid),1.000000
4,2,None,9.250000
5,2,Transparency(Low),2.000000
6,2,Transparency(Max),2.666667
7,2,Transparency(Mid),1.000000
8,2,Transparency(Min),1.000000
9,3,Black,0.500000


In [21]:
def overall_average_distance(dataframes):
    # Create a list to store the distance dataframes
    distance_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the distance for each Target in the dataframe
        distance_df = calculate_distance(df)
        # Append the resulting dataframe to the list
        distance_dfs.append(distance_df)

    # Concatenate all distance dataframes into a single dataframe
    all_data = pd.concat(distance_dfs)

    # Group by Target and calculate the average distance
    all_data = all_data.groupby('Target')['Distance'].mean().reset_index()

    return all_data

# usage
overall_avg_distances = overall_average_distance(dataframes)
overall_avg_distances

,Target,Distance
0,Black,1.006233
1,Red,1.112037
2,Size(Low),0.603797
3,Size(Max),0.911389
4,Size(Mid),0.846010
5,Transparency(Low),0.837898
6,Transparency(Max),0.727803
7,Transparency(Mid),0.658326
8,Transparency(Min),0.918708
9,White,1.059297


In [22]:
def overall_average_convergence_time(dataframes):
    # Create a list to store the convergence time dataframes
    convergence_time_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the convergence time for each 'Region' and 'Target' in the dataframe
        convergence_time_df = ConvergTime(df)
        # Append the resulting dataframe to the list
        convergence_time_dfs.append(convergence_time_df)

    # Concatenate all convergence time dataframes into a single dataframe
    all_data = pd.concat(convergence_time_dfs)

    # Group by 'Region' and 'Target' and calculate the average convergence time
    all_data = all_data.groupby(['Region', 'Target'])['ConvergenceTime'].mean().reset_index()

    return all_data

# usage
overall_avg_convergence_times = overall_average_convergence_time(dataframes)
overall_avg_convergence_times

,Region,Target,ConvergenceTime
0,1,Size(Low),0.145184
1,1,Size(Max),0.194432
2,1,Size(Mid),0.268739
3,2,Transparency(Low),0.141741
4,2,Transparency(Max),0.103894
5,2,Transparency(Mid),0.324290
6,2,Transparency(Min),0.178775
7,3,Black,0.171835
8,3,Red,0.271705
9,3,White,0.344604


In [23]:
import numpy as np

def calculate_eccentricity(dataframe):
    # Get vectors from the eye to the object and from the eye straight ahead
    dataframe = dataframe.copy()
    dataframe = dataframe[dataframe['Target'] != 'None']

    eye_to_object = dataframe[['PosX', 'PosY', 'PosZ']].values - dataframe[['PlayerPosX', 'PlayerPosY', 'PlayerPosZ']].values
    straight_ahead = np.array([0, 0, 1])  # This assumes that straight ahead is along the z-axis

    # Normalize the vectors (make them length 1) so that dot product gives the cosine of the angle
    eye_to_object = eye_to_object / np.linalg.norm(eye_to_object, axis=1, keepdims=True)
    straight_ahead = straight_ahead / np.linalg.norm(straight_ahead)

    # Calculate the dot product of the two vectors, which is equal to the cosine of the angle between them
    cos_angle = np.dot(eye_to_object, straight_ahead)

    # Calculate the angle itself (in radians)
    angle = np.arccos(cos_angle)

    # Convert to degrees
    angle = np.degrees(angle)

    # Normalize the angle by the Oculus Quest 2's field of view
    fov = 89
    angle = angle / fov

    # Add the angle to the dataframe
    dataframe['Eccentricity'] = angle

    return dataframe

calculate_eccentricity(dataframes[1])


,GazeTime,Region,Target,PosX,PosY,PosZ,PlayerPosX,PlayerPosY,PlayerPosZ,Distance,Eccentricity
4,0.055239,1,Size(Max),0.292380,0.834804,0.680269,0.000000,0.445104,0.131526,0.733805,0.467410
5,0.076259,1,Size(Max),0.267574,0.836015,0.812369,0.000000,0.421632,0.169813,0.810054,0.421483
6,0.068541,1,Size(Max),0.340478,0.823588,0.721481,0.000000,0.400537,0.204224,0.749968,0.521273
13,0.048307,1,Size(Mid),0.816961,0.709369,0.766332,0.000000,0.149450,0.613801,1.002098,0.912864
14,0.084909,1,Size(Mid),0.928217,0.649085,0.728966,0.000000,0.123317,0.656430,1.069243,0.967530
15,0.079644,1,Size(Mid),0.885648,0.686003,0.638326,0.000000,0.098803,0.696416,1.064214,1.046394
16,0.159666,1,Size(Mid),0.868910,0.675667,0.622792,0.000000,0.049660,0.776578,1.081914,1.103055
17,0.066384,1,Size(Mid),0.864217,0.649506,0.628142,0.000000,0.029230,0.809906,1.079190,1.120184
18,0.094967,1,Size(Mid),1.000000,0.577703,0.694120,0.000000,0.000000,0.857585,1.166388,1.101757
31,0.058488,2,Transparency(Low),0.420473,0.364242,0.156973,0.497900,0.991044,0.678384,0.818990,1.455534
